In [64]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import ( hamming_loss, accuracy_score,
    f1_score, classification_report, recall_score, precision_score
)
import joblib

warnings.filterwarnings("ignore")
plt.style.use('bmh')

In [65]:
def read_data(file_path):
    """
    Lee un archivo parquet y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo parquet que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo parquet.
    """
    data = pd.read_parquet(file_path)
    return data

In [66]:
def col_vectorizer_tfidf(df, col):
    """
    Convierte la columna especificada a string, la vectoriza usando TF-IDF y retorna la matriz y el vectorizador.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna a vectorizar.
    col: str
        Nombre de la columna de texto.

    Retorna
    -------
    X : scipy.sparse.csr.csr_matrix
        Matriz TF-IDF.
    vectorizer : TfidfVectorizer
        Vectorizador entrenado.
    """
    df[col] = df[col].astype(str)
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X = vectorizer.fit_transform(df[col].str.lower())
    return X, vectorizer

In [67]:
def avg_jacard(y_true,y_pred):
    """
    Calcula el porcentaje promedio de similitud de Jaccard entre matrices binarias de etiquetas verdaderas y predichas.

    Parámetros
    ----------
    y_true : array-like de forma (n_muestras, n_etiquetas)
        Matriz binaria con las etiquetas verdaderas para cada muestra.
    y_pred : array-like de forma (n_muestras, n_etiquetas)
        Matriz binaria con las etiquetas predichas para cada muestra.

    Retorna
    -------
    float
        Porcentaje promedio de Jaccard (0–100) calculado como:
        (|intersección| / |unión|) medio por muestra, multiplicado por 100.
    """
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    return jacard.mean()*100

In [68]:
def print_score(y_pred, clf):
    """
    Imprime métricas de evaluación de clasificación multilabel para un clasificador dado.

    Parámetros
    ----------
    y_pred : array-like
        Predicciones generadas por el clasificador.
    clf : objeto clasificador
        Instancia del modelo utilizado para predecir, se usa solo para mostrar su nombre de clase.

    Comportamiento
    -------------
    - Muestra por consola:
      * Nombre de la clase del clasificador.
      * Exactitud (accuracy).
      * Recall ponderado.
      * Precisión ponderada.
      * Puntuación F1 ponderada.
      * Puntuación de Jaccard promedio (porcentaje).
      * Hamming loss en porcentaje.
    - Asume que la variable global `y_test` contiene las etiquetas verdaderas.

    Retorna
    -------
    None
    """
    print("Clf: ", clf.__class__.__name__)
    print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
    print("Recall score: {}".format(recall_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("Precision score: {}".format(precision_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("F1 score: {}".format(f1_score(y_pred, y_test, average='weighted')))
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")

In [92]:
def predict_top_n_tags(text, vectorizer, classifier, multilabel_binarizer, n=5):
    """
    Predice las n etiquetas más probables para un texto dado.

    Parámetros:
    -----------
    text : str
        Texto para el cual predecir etiquetas.
    vectorizer : TfidfVectorizer
        Vectorizador TF-IDF entrenado.
    classifier : OneVsRestClassifier
        Clasificador entrenado.
    multilabel_binarizer : MultiLabelBinarizer
        Binarizador de etiquetas utilizado.
    n : int
        Número máximo de etiquetas a predecir.

    Retorna:
    --------
    list
        Lista de las n etiquetas más probables.
    """
    # Transformar el texto usando el vectorizador
    X = vectorizer.transform([text.lower()])

    # Obtener probabilidades para cada clase
    y_proba = classifier.predict_proba(X)

    # Obtener los índices de las n probabilidades más altas
    top_n_indices = y_proba[0].argsort()[-n:][::-1]

    # Crear matriz binaria para las etiquetas top
    y_pred = np.zeros(y_proba.shape[1], dtype=int)
    for idx in top_n_indices:
        # Solo incluir si la probabilidad supera un umbral mínimo (opcional)
        if y_proba[0][idx] > 0.05:
            y_pred[idx] = 1

    # Convertir de matriz binaria a etiquetas
    predicted_tags = multilabel_binarizer.inverse_transform(y_pred.reshape(1, -1))

    return predicted_tags[0]

In [69]:
data = pd.read_parquet('../data/output/StackOverflow.parquet')

In [70]:
data['text'] = data['Title'] + " " + data['Body']

X, vectorizer = col_vectorizer_tfidf(data, 'Title')

In [71]:
y = data['Tag']
multilabel_binarizer = MultiLabelBinarizer()
new_y = multilabel_binarizer.fit_transform(y)

In [72]:
new_y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1057478, 100))

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size = 0.3, random_state = 42)

In [74]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(740234, 100))

In [75]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(740234, 5000) (317244, 5000) (740234, 100) (317244, 100)


In [76]:
clf = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [77]:
y_pred = clf.predict(X_test)
print("F1-score (micro):", f1_score(y_test, y_pred, average='micro'))
print("F1-score (macro):", f1_score(y_test, y_pred, average='macro'))
print("\nReporte por etiqueta:")
print(classification_report(y_test, y_pred, target_names=multilabel_binarizer.classes_))

F1-score (micro): 0.5549179981138695
F1-score (macro): 0.49938482207380663

Reporte por etiqueta:
                    precision    recall  f1-score   support

         .htaccess       0.89      0.62      0.73      1587
              .net       0.64      0.10      0.17      7141
    actionscript-3       0.81      0.49      0.61      4702
              ajax       0.72      0.23      0.35      1990
         algorithm       0.94      0.69      0.79     27160
           android       0.96      0.71      0.82      6023
         angularjs       0.70      0.35      0.47      2005
            apache       0.41      0.07      0.13      1424
               api       0.60      0.38      0.47      5987
            arrays       0.80      0.36      0.49      8968
           asp.net       0.77      0.39      0.52      4243
       asp.net-mvc       0.86      0.45      0.59      2381
              bash       0.64      0.15      0.25      6988
                 c       0.64      0.23      0.34     30331
 

In [78]:
print_score(y_pred, clf)

Clf:  OneVsRestClassifier
Accuracy score: 0.2775434681191764
Recall score: 0.4232181591487472
Precision score: 0.7732807452320271
F1 score: 0.587956370768227
Jacard score: 42.07273536187558
Hamming loss: 1.127649380287728
---


In [110]:
new_title = "Pregunta de JS"
new_body = "¿Cómo puedo crear un carrusel de imágenes responsivo que funcione bien en móviles utilizando JavaScript y CSS? Necesito que tenga controles personalizados y transiciones suaves entre imágenes"
new_text = new_title.lower() + ' ' + new_body.lower()

# Predecir hasta 5 etiquetas más probables
predicted_tags = predict_top_n_tags(new_text, vectorizer, clf, multilabel_binarizer, n=10)
print("Etiquetas predichas para el nuevo texto:", predicted_tags)

Etiquetas predichas para el nuevo texto: ('css', 'html', 'html5', 'javascript', 'jquery', 'node.js')


In [80]:

joblib.dump(clf, '../models/modelo_logreg.joblib')
joblib.dump(vectorizer, '../models/vectorizer_tfidf.joblib')
joblib.dump(multilabel_binarizer, '../models/multilabel_binarizer.joblib')

['../models/multilabel_binarizer.joblib']

In [111]:
import pandas as pd
import numpy as np

# Cargar el dataset completo
# Si ya tienes 'data' cargado, puedes omitir esta línea
data = pd.read_parquet('../data/output/StackOverflow.parquet')

# Seleccionar 100 preguntas aleatorias
muestras_aleatorias = data.sample(n=100, random_state=42)

# Verificar que tenemos las columnas necesarias
print(f"Columnas disponibles: {muestras_aleatorias.columns.tolist()}")
print(f"Cantidad de muestras: {len(muestras_aleatorias)}")
print(f"Ejemplos de etiquetas: {muestras_aleatorias['Tag'].iloc[0]}")

# Guardar en formato parquet
ruta_salida = '../data/output/muestras_100_preguntas.parquet'
muestras_aleatorias.to_parquet(ruta_salida, index=False)

print(f"Archivo guardado en: {ruta_salida}")

Columnas disponibles: ['Id', 'Title', 'Body', 'Tag']
Cantidad de muestras: 100
Ejemplos de etiquetas: ['algorithm']
Archivo guardado en: ../data/output/muestras_100_preguntas.parquet
